In [2]:
import pandas as pd
import sklearn.metrics

In [8]:
data_reader = pd.read_csv(r'C:\Users\AdrianPan\Downloads\Yelp_Biz_User_Review.csv', usecols = ['user_id','business_id','stars_x','state'], chunksize = 10000)
data_list = []
for d in data_reader:
    d = d.loc[d['state']=='OH']
    data_list.append(d)

In [9]:
import numpy as np
cf_result = np.load(r"C:\Users\AdrianPan\Downloads\cf_result_oh.npy")

In [12]:
import sys

In [16]:
cf_result.shape

(89109, 5318)

In [17]:
data = pd.concat(data_list, axis = 0)
del data_list
data.drop('state', axis = 1,inplace = True)

In [66]:
user_id = pd.read_csv(r"C:\Users\AdrianPan\Downloads\usr_labels_OH.csv").drop("Unnamed: 0", axis = 1)['0'].to_dict()
biz_id = pd.read_csv(r"C:\Users\AdrianPan\Downloads\biz_labels_OH.csv").drop("Unnamed: 0", axis = 1)['0'].to_dict()
inv_biz_id = {val:key for key,val in biz_id.items()}

In [50]:
cf_result.shape

(89109, 5318)

In [36]:
data = data.groupby(['user_id','business_id'])['stars_x'].mean()
data = data.reset_index()

In [41]:
data.set_index('user_id', inplace = True)

In [121]:
from tqdm.notebook import tqdm
from sklearn.metrics import ndcg_score,average_precision_score

In [ ]:
RMSE = []


In [151]:
len(RMSE)

37410

In [143]:
len(data.index.unique())

89108

In [153]:
key_error_list = []
for key,usr_idx in tqdm(user_id.items()):
    if key < 45110:
        continue
    try:
        result = {}
        curr_user_actual = data.loc[[usr_idx]].set_index('business_id')
        curr_user_actual['stars_x'] = curr_user_actual['stars_x'].astype(int)
        curr_user_actual['stars_predict'] = curr_user_actual.index.map(lambda x:curr_user_pred[inv_biz_id[x]])
        if len(curr_user_actual) > 1:
            result['NDCG'] = ndcg_score([curr_user_actual['stars_x'].values], [curr_user_actual['stars_predict'].values])
        else:
            result['NDCG'] = np.nan

        curr_user_actual = (curr_user_actual-curr_user_actual.min())/(curr_user_actual.max()-curr_user_actual.min())
        curr_user_pred = (curr_user_pred-curr_user_pred.min())/(curr_user_pred.max()-curr_user_pred.min())
    
    
        result['SE'] = np.sum((curr_user_actual['stars_x']-curr_user_actual['stars_predict'])**2)
        result['count'] = len(curr_user_actual)
        RMSE.append(
            result
        )
    except Exception as e:
        key_error_list.append(key)
        print(key,":",e)
        
        if len(key_error_list) > 5:
            break
        
        continue

  0%|          | 0/89109 [00:00<?, ?it/s]

45110 : "None of [Index(['VjfIomGho3uyPRah9Clmlw'], dtype='object', name='user_id')] are in the [index]"


In [174]:
result = pd.DataFrame(RMSE)
result = result[result['NDCG'].notnull()]

In [179]:
data

,business_id,stars_x
user_id,,
--1_pDM1pQ26cqhLx9agWQ,qa4SegtG2bWMBhJgWyU7Tw,5.0
--2PnhMMH7EYoY3wywOvgQ,32AcG_zpsPzMgo0aWj7zQA,4.0
--2PnhMMH7EYoY3wywOvgQ,8lS-sVYxXqVbhV8vjitzDg,4.0
--2PnhMMH7EYoY3wywOvgQ,AEzIqFtXrJITE4toGNtAuA,3.0
--2PnhMMH7EYoY3wywOvgQ,IHCD--427ou0ODW6JRoYpw,4.0
...,...,...
zzoLFpcO9IvQV_zIt7A1rA,9xXvZsc0f0UguKevFaUikA,1.0
zzoru1fhdVzDIUGd59y1Sg,Ht3Sew5RcND7Q_La0gbG1A,5.0
zzvQiBdoT9Vd72h4Eh0_Bg,AHMA7314bg975Gy1JxZDkg,5.0


In [175]:
np.sqrt(result['SE'].sum() / result['count'].sum())

0.5646631503456211

In [177]:
np.dot(result['NDCG'], result['count'])/result['count'].sum()

0.9222932937741084